In [1]:
!pip install torchmetrics

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import os
import numpy as np
import pandas as pd


In [ ]:
# from google.colab import drive

# drive.mount("/content/drive")

# TEST_IMG_PATH = "/content/drive/MyDrive/AI/Cityscape/images_prepped_test-20230811T065241Z-001"
# TEST_MASK_PATH = "/content/drive/MyDrive/AI/Cityscape/annotations_prepped_test-20230811T065240Z-001"

# EDA_DS_PATH = "/content/drive/MyDrive/AI/Advance CV - Project 3/Dataset"
# EDA_TRAIN_LABELS = os.path.join(EDA_DS_PATH, "train_labels.csv")
# EDA_TRAIN_FILES = os.path.join(EDA_DS_PATH, "train_files.csv")
# EDA_VAL_FILES = os.path.join(EDA_DS_PATH, "val_files.csv")

# MODEL_PATH = "/content/drive/MyDrive/AI/Advance CV - Project 3/models/UNet"
# MODEL_PATH_FCN8S = "/content/drive/MyDrive/AI/Advance CV - Project 3/models/FCN8s"

In [4]:
TEST_IMG_PATH = "./dataset/bootcamp/test/images"
TEST_MASK_PATH = "./dataset/bootcamp/test/annotations"

EDA_DS_PATH = "./dataset/bootcamp/train"
EDA_TRAIN_LABELS = os.path.join(EDA_DS_PATH, "train_labels.csv")
EDA_TRAIN_FILES = os.path.join(EDA_DS_PATH, "train_files.csv")
EDA_VAL_FILES = os.path.join(EDA_DS_PATH, "val_files.csv")

MODEL_PATH = "./models/UNet"
MODEL_PATH_FCN8S = "./models/FCN8s"

In [5]:
# use validation images for inference test
labels_df = pd.read_csv(EDA_TRAIN_LABELS)

image_files = os.listdir(TEST_IMG_PATH)
mask_files = os.listdir(TEST_MASK_PATH)

test_files = [x for x in image_files if x in mask_files]

print(f"Test Files: {len(test_files)}, Image: {len(image_files)}, Mask: {len(mask_files)}")

print(labels_df)

Test Files: 101, Image: 101, Mask: 101
    class         label
0       0           sky
1       1      building
2       2          pole
3       3          road
4       4      sidewalk
5       5    vegetation
6       6  traffic sign
7       7         fence
8       8       vehicle
9       9        person
10     10         rider
11     11          void


## Dataset Loading

In [6]:
from dataset import CityScapes
from torch.utils.data import DataLoader

test_datasets = CityScapes(TEST_IMG_PATH, TEST_MASK_PATH, test_files)
test_dataloader = DataLoader(test_datasets, batch_size=32, shuffle=True, num_workers=0)

## Engine

In [25]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

    
def unnormalize(tensor, mean, std):
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

def visualize_results(images, preds, masks, labels_df):
    """Visualize batch results"""
    n_classes = len(labels_df)
    legend_colors = [plt.cm.nipy_spectral(i / (n_classes - 1)) for i in range(n_classes)]
    legend_labels = [f"{labels_df['label'][i]}" for i in range(n_classes)]
    patches = [mpatches.Patch(color=legend_colors[i], label=legend_labels[i]) for i in range(n_classes)]
    
    mean = [0.485, 0.456, 0.406]
    std  = [0.229, 0.224, 0.225]
    
    fig, axs = plt.subplots(3, 4, figsize=(20, 18))
    for i in range(4):  # Show first 4 samples
        # print(f"{i}: Min: {images[i].min()}, Max: {images[i].max()}")
        
        img = unnormalize(images[i].clone(), mean, std)
        # Input image
        axs[0,i].imshow(img.permute(1,2,0).cpu().numpy())
        axs[0,i].set_title(f"Input {i}")
        axs[0,i].axis('off')
        
        # Prediction
        axs[1,i].imshow(preds[i].cpu().numpy(), 
                       cmap='nipy_spectral', vmin=0, vmax=n_classes-1)
        axs[1,i].set_title(f"Pred {i}")
        axs[1,i].axis('off')
        
        # Ground Truth
        axs[2,i].imshow(masks[i].cpu().numpy(), 
                       cmap='nipy_spectral', vmin=0, vmax=n_classes-1)
        axs[2,i].set_title(f"GT {i}")
        axs[2,i].axis('off')
    fig.legend(handles=patches, loc='lower center', ncol=n_classes, fontsize='large', frameon=False)
    fig.subplots_adjust(bottom=0.15)
    plt.tight_layout()
    plt.show()

SyntaxError: '(' was never closed (1966150826.py, line 9)

In [43]:
from torchmetrics.functional.segmentation import dice_score
from torchmetrics import JaccardIndex
import time

# def inferences(model, test_dataloader, labels_df):
#     total_time = 0
#     num_classes = len(labels_df)
#     all_dice_scores = []
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device).eval()  # ✅ Model to device + eval mode

#     # Warm-up (avoid CUDA init overhead)
#     with torch.no_grad():
#         warmup = next(iter(test_dataloader))
#         _ = inference(warmup[0].to(device), model)

#     # Initialize metrics
#     jaccard = JaccardIndex(task='multiclass', num_classes=num_classes).to(device)
    
#     with torch.no_grad():
#         for batch, (images, masks) in enumerate(test_dataloader, 1):
#             # Move data to device (correctly)
#             images, masks = images.to(device), masks.to(device)
            
#             # Time inference with synchronization
#             start = torch.cuda.Event(enable_timing=True)
#             end = torch.cuda.Event(enable_timing=True)
#             start.record()
#             logits, probs, preds, _ = inference(images, model)
#             end.record()
#             torch.cuda.synchronize()
#             inf_time = start.elapsed_time(end) / 1000  # Seconds
            
#             # Compute metrics
#             dice = dice_score(preds, masks, num_classes=num_classes, average='macro', input_format='index')
#             all_dice_scores.extend(dice.cpu().tolist())
#             jaccard.update(preds, masks)
#             total_time += inf_time

#     # Final metrics
#     avg_dice = torch.mean(torch.tensor(all_dice_scores))
#     mIoU = jaccard.compute()

#     out_iou_per_class = mIoU.cpu().numpy()
    
#     print(f"\nDataset Metrics:")
#     print(f"- Per-class IoU: {out_iou_per_class}")
#     print(f"- Mean IoU (mIoU): {mIoU.item():.4f}")
#     print(f"- Average Dice: {avg_dice:.4f}")
#     print(f"- Total Inference Time: {total_time:.4f}s")
    
#     return (
#         out_iou_per_class,  # Per-class IoU
#         mIoU.item(),                      # Mean IoU
#         avg_dice.item(),                  # Avg Dice
#         total_time                        # Total time
#     )


def inference(batch_tensor, model, device='cuda'):
    """Proper batch inference function that handles all caases"""
    
    #GPU-optimized inference with accurate timing.
    # Initialize CUDA events
    start_event = torch.cuda.Event(enable_timing=True)
    end_event = torch.cuda.Event(enable_timing=True)

    with torch.no_grad():
        # Ensure correct input dimensions
        if batch_tensor.dim() == 3:  # Single image (C,H,W)
            batch_tensor = batch_tensor.unsqueeze(0)  # Add batch dim
        elif batch_tensor.dim() != 4:  # Not standard 4D input
            raise ValueError(f"Expected 3D or 4D input, got {batch_tensor.dim()}D")
        
        batch_tensor = batch_tensor.to(device)
        
        # Synchronize and record time
        torch.cuda.synchronize()  # Wait for pending GPU ops
        start_event.record()
        logits = model(batch_tensor)
        probs = torch.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1)
        end_event.record()
        torch.cuda.synchronize()  # Ensure all work is done
        inf_time = start_event.elapsed_time(end_event) / 1000  # Convert ms to seconds

    return logits, probs, preds, inf_time
    
def inferences(model, test_dataloader, labels_df):
    """
    input:
    - model: model
    - test_dataloader: dataloader
    """
    
    total_time = 0
    num_classes = len(labels_df)
    all_dice_scores = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Initialize metrics
    jaccard = JaccardIndex(
        task='multiclass',
        num_classes=num_classes,
        ignore_index=None,
        average='none'  # Track per-class IoU
    ).to(device)
    batch = 0
    model.to(device)
    model.eval()
    with torch.no_grad():
        for images, masks in test_dataloader:
            batch = batch + 1
            print(f"Batch Size: {len(images)}, batch: {batch} of {len(test_dataloader)}")
            images, masks = images.to(device), masks.to(device)
            # Verify shapes
            if batch == 1:
                print("Input shape:", images.shape)  # Should be [B,C,H,W]
                print("Mask shape:", masks.shape)   # Should be [B,H,W]
            
            # Run inference
            logits, probs, preds, inf_time = inference(images, model)
            print(f"Inf time:{inf_time:.4f}")
            preds = preds.to(device).long()
            masks = masks.to(device).long()
            
            # Update Dice (per batch)
            # input_format='index' if masks.shape is [B, H, W]
            # if mask [B, C, H, W] input format='probabilities'
            dice = dice_score(preds, masks, num_classes=num_classes, average='micro', input_format='index')
            all_dice_scores.extend(dice.cpu().tolist())
            
            # Update IoU (accumulate across batches)
            jaccard.update(preds, masks)
            
            total_time += inf_time
            print()
            
    # Compute final metrics
    iou_per_class = jaccard.compute()  # IoU per class [num_classes]
    mIoU = torch.mean(iou_per_class)   # Mean IoU (scalar)
    # print("Raw Dice scores:", all_dice_scores)
    avg_dice = sum(all_dice_scores) / len(all_dice_scores)

    out_iou_per_class = iou_per_class.cpu().numpy().round(4)
    out_mean_iou = mIoU.item()
    print(f"\nDataset Metrics:")
    print(f"- Per-class IoU: {out_iou_per_class}")
    print(f"- Mean IoU (mIoU): {out_mean_iou:.4f}")
    print(f"- Average Dice: {avg_dice:.4f}")
    print(f"- Total Inference Time: {total_time:.4f}s")
    
    return out_iou_per_class, out_mean_iou, avg_dice, total_time

## Inference

In [44]:
metrics = []

In [45]:
from architecture import UNET
import torch
from PIL import Image

model = UNET(in_channels=3, out_channels=len(labels_df))
state_dict = torch.load(os.path.join(MODEL_PATH, 'best_model_dice.pth'), weights_only=True)
missing, unexpected = model.load_state_dict(state_dict)
# print("Missing keys:", missing)

iou_per_class, mean_iou, avg_dice, total_time = inferences(model,test_dataloader,labels_df)
metrics.append({
    "Architecture": "UNet",
    "Loss Function": "Dice Loss",
    "IoU_per_class": iou_per_class,
    "Mean_IoU": mean_iou,
    "Avg_Dice": avg_dice,
    "Total_Time": total_time
})


Batch Size: 32, batch: 1 of 4
Input shape: torch.Size([32, 3, 256, 256])
Mask shape: torch.Size([32, 256, 256])
Inf time:1.1070

Batch Size: 32, batch: 2 of 4
Inf time:0.8233

Batch Size: 32, batch: 3 of 4
Inf time:1.3307

Batch Size: 5, batch: 4 of 4
Inf time:0.1054


Dataset Metrics:
- Per-class IoU: [0.9371 0.8088 0.1542 0.9452 0.8363 0.8372 0.3528 0.     0.577  0.3909
 0.     0.2537]
- Mean IoU (mIoU): 0.5078
- Average Dice: 0.8801
- Total Inference Time: 3.3665s


In [46]:
model = UNET(in_channels=3, out_channels=len(labels_df))
state_dict = torch.load(os.path.join(MODEL_PATH, 'best_model_cross.pth'), weights_only=True)
missing, unexpected = model.load_state_dict(state_dict)
# print("Missing keys:", missing)

iou_per_class, mean_iou, avg_dice, total_time = inferences(model,test_dataloader,labels_df)
metrics.append({
    "Architecture": "UNet",
    "Loss Function": "Cross Entropy Loss",
    "IoU_per_class": iou_per_class,
    "Mean_IoU": mean_iou,
    "Avg_Dice": avg_dice,
    "Total_Time": total_time
})

Batch Size: 32, batch: 1 of 4
Input shape: torch.Size([32, 3, 256, 256])
Mask shape: torch.Size([32, 256, 256])
Inf time:0.8496

Batch Size: 32, batch: 2 of 4
Inf time:0.8495

Batch Size: 32, batch: 3 of 4
Inf time:1.3685

Batch Size: 5, batch: 4 of 4
Inf time:0.1116


Dataset Metrics:
- Per-class IoU: [0.9343 0.8452 0.0763 0.9686 0.8672 0.8838 0.3131 0.5895 0.6708 0.3183
 0.5661 0.2247]
- Mean IoU (mIoU): 0.6048
- Average Dice: 0.9132
- Total Inference Time: 3.1792s


In [47]:
model = UNET(in_channels=3, out_channels=len(labels_df))
state_dict = torch.load(os.path.join(MODEL_PATH, 'best_model_combined.pth'), weights_only=True)
missing, unexpected = model.load_state_dict(state_dict)
# print("Missing keys:", missing)

iou_per_class, mean_iou, avg_dice, total_time = inferences(model,test_dataloader,labels_df)
metrics.append({
    "Architecture": "UNet",
    "Loss Function": "Dice Loss + Cross Entropy Loss",
    "IoU_per_class": iou_per_class,
    "Mean_IoU": mean_iou,
    "Avg_Dice": avg_dice,
    "Total_Time": total_time
})

Batch Size: 32, batch: 1 of 4
Input shape: torch.Size([32, 3, 256, 256])
Mask shape: torch.Size([32, 256, 256])
Inf time:0.8471

Batch Size: 32, batch: 2 of 4
Inf time:0.8474

Batch Size: 32, batch: 3 of 4
Inf time:1.3494

Batch Size: 5, batch: 4 of 4
Inf time:0.1098


Dataset Metrics:
- Per-class IoU: [0.938  0.8104 0.0988 0.968  0.8339 0.8753 0.3573 0.5182 0.618  0.3382
 0.5597 0.2165]
- Mean IoU (mIoU): 0.5944
- Average Dice: 0.9031
- Total Inference Time: 3.1537s


In [48]:
from architecture import FCN8s
import torch

model = FCN8s(in_channels=3, out_channels=len(labels_df))
state_dict = torch.load(os.path.join(MODEL_PATH_FCN8S, 'best_model_dice.pth'), weights_only=True)
missing, unexpected = model.load_state_dict(state_dict)
# print("Missing keys:", missing)

inferences(model,test_dataloader,labels_df)
iou_per_class, mean_iou, avg_dice, total_time = inferences(model,test_dataloader,labels_df)
metrics.append({
    "Architecture": "FCN8s",
    "Loss Function": "Cross Entropy Loss",
    "IoU_per_class": iou_per_class,
    "Mean_IoU": mean_iou,
    "Avg_Dice": avg_dice,
    "Total_Time": total_time
})

Batch Size: 32, batch: 1 of 4
Input shape: torch.Size([32, 3, 256, 256])
Mask shape: torch.Size([32, 256, 256])
Inf time:0.3424

Batch Size: 32, batch: 2 of 4
Inf time:0.3365

Batch Size: 32, batch: 3 of 4
Inf time:0.3434

Batch Size: 5, batch: 4 of 4
Inf time:0.0510


Dataset Metrics:
- Per-class IoU: [0.9269 0.7861 0.1391 0.9366 0.8121 0.8435 0.3302 0.     0.6973 0.2852
 0.     0.272 ]
- Mean IoU (mIoU): 0.5024
- Average Dice: 0.8787
- Total Inference Time: 1.0733s
Batch Size: 32, batch: 1 of 4
Input shape: torch.Size([32, 3, 256, 256])
Mask shape: torch.Size([32, 256, 256])
Inf time:0.3376

Batch Size: 32, batch: 2 of 4
Inf time:0.3477

Batch Size: 32, batch: 3 of 4
Inf time:0.3342

Batch Size: 5, batch: 4 of 4
Inf time:0.0520


Dataset Metrics:
- Per-class IoU: [0.9269 0.7861 0.1391 0.9366 0.8121 0.8435 0.3302 0.     0.6973 0.2852
 0.     0.272 ]
- Mean IoU (mIoU): 0.5024
- Average Dice: 0.8787
- Total Inference Time: 1.0715s


In [49]:
from architecture import FCN8s
import torch

model = FCN8s(in_channels=3, out_channels=len(labels_df))
state_dict = torch.load(os.path.join(MODEL_PATH_FCN8S, 'best_model_cross.pth'), weights_only=True)
missing, unexpected = model.load_state_dict(state_dict)
# print("Missing keys:", missing)

iou_per_class, mean_iou, avg_dice, total_time = inferences(model,test_dataloader,labels_df)
metrics.append({
    "Architecture": "FCN8s",
    "Loss Function": "Cross Entropy Loss",
    "IoU_per_class": iou_per_class,
    "Mean_IoU": mean_iou,
    "Avg_Dice": avg_dice,
    "Total_Time": total_time
})

Batch Size: 32, batch: 1 of 4
Input shape: torch.Size([32, 3, 256, 256])
Mask shape: torch.Size([32, 256, 256])
Inf time:0.3355

Batch Size: 32, batch: 2 of 4
Inf time:0.3396

Batch Size: 32, batch: 3 of 4
Inf time:0.3408

Batch Size: 5, batch: 4 of 4
Inf time:0.0535


Dataset Metrics:
- Per-class IoU: [0.8707 0.74   0.     0.7368 0.0332 0.749  0.     0.     0.2407 0.
 0.     0.0051]
- Mean IoU (mIoU): 0.2813
- Average Dice: 0.7703
- Total Inference Time: 1.0693s


In [50]:
from architecture import FCN8s
import torch

model = FCN8s(in_channels=3, out_channels=len(labels_df))
state_dict = torch.load(os.path.join(MODEL_PATH_FCN8S, 'best_model_combined.pth'), weights_only=True)
missing, unexpected = model.load_state_dict(state_dict)
# print("Missing keys:", missing)

iou_per_class, mean_iou, avg_dice, total_time = inferences(model,test_dataloader,labels_df)
metrics.append({
    "Architecture": "FCN8s",
    "Loss Function": "Dice Loss + Cross Entropy Loss",
    "IoU_per_class": iou_per_class,
    "Mean_IoU": mean_iou,
    "Avg_Dice": avg_dice,
    "Total_Time": total_time
})

Batch Size: 32, batch: 1 of 4
Input shape: torch.Size([32, 3, 256, 256])
Mask shape: torch.Size([32, 256, 256])
Inf time:0.3340

Batch Size: 32, batch: 2 of 4
Inf time:0.3383

Batch Size: 32, batch: 3 of 4
Inf time:0.3411

Batch Size: 5, batch: 4 of 4
Inf time:0.0510


Dataset Metrics:
- Per-class IoU: [0.9266 0.8297 0.058  0.9521 0.8267 0.8911 0.3114 0.5265 0.7375 0.2668
 0.4567 0.1472]
- Mean IoU (mIoU): 0.5775
- Average Dice: 0.9005
- Total Inference Time: 1.0643s


In [51]:
df_metrics = pd.DataFrame(metrics)
df_metrics.to_csv("inference_metrics.csv", index=False)

In [ ]:
model = UNET(in_channels=3, out_channels=len(labels_df))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
state_dict = torch.load(os.path.join(MODEL_PATH, 'best_model_combined.pth'), weights_only=True)
missing, unexpected = model.load_state_dict(state_dict)
print("Missing keys:", missing)
print("Unexpected keys:", unexpected)

# Usage with DataLoader
model.eval()
with torch.no_grad():
    for images, masks in test_dataloader:
        # Verify shapes
        print("Input shape:", images.shape)  # Should be [B,C,H,W]
        print("Mask shape:", masks.shape)   # Should be [B,H,W]
        
        # Run inference
        logits, probs, preds = inference(images, model)
        iou_score = calculate_iou_scores(masks, preds)
        print('masks:')
        print(masks)
        print()
        print('preds: ')
        print(preds)
        # Verify output shapes
        print("Logits shape:", logits.shape)  # Should be [B,C,H,W]
        print("Preds shape:", preds.shape)    # Should be [B,H,W]
        print(f"IOU score: {iou_score}")
        print(f"IOU score mean: {np.mean(iou_score)}")
        dice_score = calculate_dice_coefficients(masks, preds)
        print(f"Dice Coef score: {iou_score}")
        print(f"Dice Coef score mean: {np.mean(iou_score)}")
        
        # Visualize
        visualize_results(images, preds, masks, labels_df)
        
        break  # Only show first batch